Restart cuda module after suspend by:  
`$ sudo rmmod nvidia_uvm`  
`$ sudo modprobe nvidia_uvm`

1. Test용으로 몇 장 빼둘 것. 
2. Train의 목적: -- T-type을 N개의 label로 잘 구별하는 representation vector를 구하는 것.  
3. 목적대로 train 되었는지 testset으로 확인 -- Freeze된 ResNet으로 classification을 잘 수행하는가..  
    3.1 ResNet은 아무런 train이 필요 없나? 뭔갈 해야하나? 
4. train 잘 되었다면 그 담에 뭘 할까? -- M20, Gini, Contrast등의 값을 feature vector로 변경. --> custom_morph 코드를 Train된 ResNet으로 변경. 

ToneMapping이 바뀔 때마다 ResNet의 마지막 layer 정도는 fine-tune할 수 있지 않을까? 

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
from torchvision.transforms import transforms
from torchvision import models

import atm
import atm.simclr as simclr

In [4]:
import argparse 

args = argparse.Namespace()

args.data='./datasets' 
args.dataset_name='cifar10'
args.arch='resnet50'
args.workers=1
args.epochs=300 
args.batch_size=256 
args.lr=0.02
args.wd=0.0005
args.disable_cuda=False
args.fp16_precision=True
args.out_dim=10
args.log_every_n_steps=100
args.temperature=0.07
args.n_views = 2
args.gpu_index=0
args.device='cuda' if torch.cuda.is_available() else 'cpu'

print("Using device:", args.device)

assert args.n_views == 2, "Only two view training is supported. Please use --n-views 2."
# check if gpu training is available
if not args.disable_cuda and torch.cuda.is_available():
    args.device = torch.device('cuda')
    cudnn.deterministic = True
    cudnn.benchmark = True
else:
    args.device = torch.device('cpu')
    args.gpu_index = -1

Using device: cuda


# Data Loader

In [5]:
import matplotlib.pyplot as plt 

from PIL import Image
from typing import Any, Callable, Optional, Tuple
from torchvision.datasets.vision import VisionDataset
from functools import partial
from astrobf.tmo import Mantiuk_Seidel

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


## Original 3-channel version

In [7]:
from torchvision.transforms import transforms
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
#from exceptions.exceptions import InvalidDatasetSelection
from atm.simclr.utils import GaussianBlur

In [18]:
class ContrastiveLearningDataset:
    def __init__(self, root_folder):
        self.root_folder = root_folder

    @staticmethod
    def get_simclr_pipeline_transform(size, s=1):
        """Return a set of data augmentation transformations as described in the SimCLR paper."""
        color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
        data_transforms = transforms.Compose([transforms.RandomResizedCrop(size=size),
                                              transforms.RandomHorizontalFlip(),
                                              transforms.RandomApply([color_jitter], p=0.8),
                                              transforms.RandomGrayscale(p=0.2),
                                              GaussianBlur(kernel_size=int(0.1 * size)),
                                              transforms.ToTensor(),
                                              transforms.Lambda(lambda x: x.mean(dim=0, keepdim=True))])
                                               # merge as 1 channel.
        return data_transforms

    def get_dataset(self, name, n_views):
        valid_datasets = {'cifar10': lambda: datasets.CIFAR10(self.root_folder, train=True,
                                                              transform=ContrastiveLearningViewGenerator(
                                                                  self.get_simclr_pipeline_transform(32),
                                                                  n_views),
                                                              download=True),

                          'stl10': lambda: datasets.STL10(self.root_folder, split='unlabeled',
                                                          transform=ContrastiveLearningViewGenerator(
                                                              self.get_simclr_pipeline_transform(96),
                                                              n_views),
                                                          download=True)}

        dataset_fn = valid_datasets[name]
        return dataset_fn()

In [19]:
dataset = ContrastiveLearningDataset(args.data)

train_dataset = dataset.get_dataset(args.dataset_name, args.n_views)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True, drop_last=True)

Files already downloaded and verified


In [21]:
from tqdm import tqdm
from torch.cuda.amp import autocast
import importlib

In [23]:
# Make sure this is the same as the original version!!!!!!!!!!!!!11
importlib.reload(simclr.models)
model = simclr.models.ResNetSimCLR(base_model=args.arch, out_dim=args.out_dim, num_channels=3)

optimizer = torch.optim.Adam(model.parameters(), args.lr, weight_decay=args.wd)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0,
                                                           last_epoch=-1)

In [24]:
np.seterr(divide='ignore')

with torch.cuda.device(args.gpu_index):
    simc = simclr.models.SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, args=args)
    simc.train(train_loader) # model is saved at the end of train.

  0%|          | 0/195 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[512, 1, 32, 32] to have 3 channels, but got 1 channels instead

class SimCLR는 원본과 다른게 없음. 

class ResNetSimCLR도 별로 다른게 없으나, ResNet 만드는 부분의 코드가 어떤지 몰겠네. 결과물 확인 필요. 

## 1-channel 

특정 배치에 있었던 이미지를 보려면?

loss는 쬐끔 줄어드려던 중이었음. weight가 explode한 듯? 

In [15]:

img_size = 32

# view generator
class ContrastiveLearningViewGenerator(object):
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform, n_views=2):
        self.base_transform = base_transform
        self.n_views = n_views

    def __call__(self, x):
        return [self.base_transform(x) for i in range(self.n_views)]


class ContrastiveLearningDataset:
    def __init__(self, root_folder):
        self.root_folder = root_folder

    @staticmethod 
    def get_simclr_pipeline_transform(size, s=1):
        """Return a set of data augmentation transformations as described in the SimCLR paper."""
        color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
        data_transforms = transforms.Compose([transforms.RandomResizedCrop(size),
                                              transforms.RandomHorizontalFlip(p=0.5),
                                              #transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8), 
                                              #transforms.RandomGrayscale(p=0.2), 
                                              GaussianBlur(kernel_size=int(0.1 * size)),
                                              transforms.ToTensor(),
                                              transforms.Lambda(lambda x: x.mean(dim=0, keepdim=True)), # 1-channel
                                              transforms.Normalize([0.5], [0.2])])
        return data_transforms

    def get_dataset(self, dataset_name = 'cifar10', n_views=2):
        if dataset_name == 'cifar10':
            dataset_fn = lambda: datasets.CIFAR10(self.root_folder, 
                                              train=True,
                                              transform=ContrastiveLearningViewGenerator(
                                                  self.get_simclr_pipeline_transform(32), # image size = 32
                                                  n_views),
                                              download=True)
        else:
            raise NotImplementedError('Only cifar10 dataset is supported')

        return dataset_fn()

In [13]:
np.seterr(divide='ignore')

with torch.cuda.device(args.gpu_index):
    simc = simclr.models.SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, args=args)
    simc.train(train_loader) # model is saved at the end of train.

  0%|          | 0/195 [00:00<?, ?it/s]

current loss tensor(7.2900, device='cuda:0', grad_fn=<NllLossBackward>)


/home/hoseung/anaconda3/envs/tm39/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:508: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
  1%|          | 1/195 [00:02<07:17,  2.26s/it]

current loss tensor(7.3008, device='cuda:0', grad_fn=<NllLossBackward>)
current loss tensor(7.2341, device='cuda:0', grad_fn=<NllLossBackward>)


  2%|▏         | 4/195 [00:02<01:38,  1.95it/s]

...

 12%|█▏        | 24/195 [00:07<00:46,  3.69it/s]

current loss tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)


 13%|█▎        | 25/195 [00:08<00:57,  2.97it/s]

current loss tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)


KeyboardInterrupt: 

In [17]:
np.seterr(divide='ignore')

with torch.cuda.device(args.gpu_index):
    simc = simclr.models.SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, args=args)
    simc.train(train_loader) # model is saved at the end of train.

  0%|          | 0/14 [00:00<?, ?it/s]

feature min max -0.614746 0.624023
current loss tensor(6.9046, device='cuda:0', grad_fn=<NllLossBackward>)


  7%|▋         | 1/14 [00:01<00:16,  1.27s/it]

feature min max -0.616699 0.629883
current loss tensor(6.8819, device='cuda:0', grad_fn=<NllLossBackward>)


 14%|█▍        | 2/14 [00:01<00:08,  1.38it/s]

...

feature min max -inf inf
current loss tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)


 93%|█████████▎| 13/14 [00:05<00:00,  2.85it/s]

feature min max -inf inf
current loss tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)


 93%|█████████▎| 13/14 [00:05<00:00,  2.24it/s]


KeyboardInterrupt: 

# representation quality check

## linear evaluation protocol, a standard way
Train a linear classifier on the fixed representations

## Load checkpoint

In [33]:
if True:
    checkpoint = torch.load('./runs/Aug23_00-13-48_hoseung/checkpoint_0300.pth.tar', map_location=args.device)
    state_dict = checkpoint['state_dict']
    
    resnet = simclr.models.ResNetSimCLR(base_model=args.arch, out_dim=args.out_dim)
    resnet.to(args.device)
    log = resnet.load_state_dict(state_dict, strict=False)

    print(log)
else:
    resnet = simc.model
    print("Using live simc instance")



<All keys matched successfully>


### Discard the projection head and leave the original backbone. 

In [21]:
# drop the projection head
resnet.backbone.fc = nn.Identity()

## t_SNE 

### Get features of the test set

test_transform에서 to_tensor해줬는데...??

### Test set Evaluation

In [34]:
from tqdm import tqdm
from torch.cuda.amp import autocast
feature_arr = []
for images, _ in tqdm(test_loader):
    #images = torch.cat(images, dim=0)
    images = images.to(args.device)

    # autocast <- AMP
    with autocast(enabled=args.fp16_precision):
        features = resnet.backbone(images)
        
    feature_arr.append(features.cpu().detach().numpy())

feature_arr = np.concatenate(feature_arr)

100%|██████████| 3/3 [00:01<00:00,  2.77it/s]


In [43]:
resnet.backbone.parameters()

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 2

In [68]:
b1 = resnet.backbone.layer4[0]

In [70]:
b1

Bottleneck(
  (conv1): Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [72]:
b1.conv1.weight

Parameter containing:
tensor([[[[nan]],

         [[nan]],

         [[nan]],

         ...,

         [[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]],

         ...,

         [[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]],

         ...,

         [[nan]],

         [[nan]],

         [[nan]]],


        ...,


        [[[nan]],

         [[nan]],

         [[nan]],

         ...,

         [[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]],

         ...,

         [[nan]],

         [[nan]],

         [[nan]]],


        [[[nan]],

         [[nan]],

         [[nan]],

         ...,

         [[nan]],

         [[nan]],

         [[nan]]]], device='cuda:0', requires_grad=True)

In [21]:
from sklearn.manifold import TSNE

In [54]:
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature_arr)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# 2D projection of 1024(?)-dim features
tsne = TSNE(n_components=2).fit_transform(features)


In [11]:
optimizer = torch.optim.Adam(resnet.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(args.device)

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [12]:
from torch.utils.data import DataLoader
def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
    train_dataset = torchvision.datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)

    test_dataset = torchvision.datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

    test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
    return train_loader, test_loader

train_loader, test_loader = get_cifar10_data_loaders(download=True)
# num_workers=0. Using multiple (even num_workers=1) causes an error 'can only test a child process'.

Files already downloaded and verified
Files already downloaded and verified


In [120]:
epochs = 100
for epoch in range(epochs):
    top1_train_accuracy = 0
    for counter, (x_batch,y_batch) in enumerate(train_loader):
        x_batch = x_batch.to(args.device)
        y_batch = y_batch.to(args.device)
        
        logits = resnet(x_batch)
        loss = criterion(logits, y_batch)
        top1 = accuracy(logits, y_batch, topk=(1,))
        top1_train_accuracy += top1[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    top1_train_accuracy /= (counter + 1)
    top1_accuracy = 0
    top5_accuracy = 0
    for counter, (x_batch, y_batch) in enumerate(test_loader):
        x_batch = x_batch.to(args.device)
        y_batch = y_batch.to(args.device)

        logits = resnet(x_batch)

        top1, top5 = accuracy(logits, y_batch, topk=(1,5))
        top1_accuracy += top1[0]
        top5_accuracy += top5[0]

    top1_accuracy /= (counter + 1)
    top5_accuracy /= (counter + 1)
    print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

KeyboardInterrupt: 

In [ ]:
class SimCLRClassifier(nn.Module):
    def __init__(self, base_model, freeze_base, base_feature_size=512, n_views=2):
        self.embeddings = base_model
        
        if freeze_base:
            print("Freezing embeddings")
            for param in self.embeddings.parameters():
                param.requires_grad = False
                
        self.classifier = nn.Linear(in_features = base_feature_size, 
                                   out_features = n_views) #
    
    def forward(self, X, *args):
        emb = self.embeddings(X)
        return self.classifier(emb)
    
class SimCLRClassifierModule

In [63]:
from sklearn.metrics import classification_report

In [ ]:
def evaluate(data_loader, module):
    with torch.no_grad():
        progress = ["/", "-", "\\", "|", "/", "-", "\\", "|"]
        module.eval().cuda()
        true_y, pred_y = [], []
        for i, batch_ in enumerate(data_loader):
            X, y = batch_
            print(progress[i % len(progress)], end="\r")
            y_pred = torch.argmax(module(X.cuda()), dim=1)
            true_y.extend(y.cpu())
            pred_y.extend(y_pred.cpu())
        print(classification_report(true_y, pred_y, digits=3))
        return true_y, pred_y

_ = evaluate(module.val_dataloader(), module)